<a href="https://colab.research.google.com/github/rohitratannagar/AgentsByRohit/blob/main/GitHub_Repo_Reviewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
from IPython.display import display, Markdown
import gradio as gr

In [ ]:
from openai import OpenAI
from google.colab import userdata

In [ ]:
github_token = userdata.get('GITHUB_TOKEN')
github_username = "rohitratannagar"
gemini_api_key = userdata.get('GROQ_API_KEY')

In [ ]:
print(f"gemii api key starts {gemini_api_key[:5]}")
print(f"github token starts {github_token[:5]}")

gemii api key starts gsk_v
github token starts githu


In [ ]:
get_ipython().system('pip install PyGithub')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.6 MB/s eta 0:00:00


In [ ]:
from github import Github, Auth
github = Github(auth=Auth.Token(github_token))

# repo = g.get_repo(f"{github_username}/talksy")

# for file in repo.get_contents(""):
#     print(file.path)

In [ ]:
def fetch_repo_files(repo_name):
    repo = github.get_repo(repo_name)
    files = []
    contents = repo.get_contents("")
    while contents:
        file = contents.pop(0)
        if file.type == "file":
            files.append(file.path)
        else:
            contents.extend(repo.get_contents(file.path))
    return files

In [ ]:
fetch_repo_files("rohitratannagar/talksy")

['.gitignore',
 'README.md',
 'package-lock.json',
 'package.json',
 'backend/package-lock.json',
 'backend/package.json',
 'frontend/README.md',
 'frontend/eslint.config.js',
 'frontend/index.html',
 'frontend/package-lock.json',
 'frontend/package.json',
 'frontend/postcss.config.js',
 'frontend/tailwind.config.js',
 'frontend/vite.config.js',
 'backend/src/index.js',
 'frontend/public/Nouser.webp',
 'frontend/public/avatar.png',
 'frontend/public/talksy.svg',
 'frontend/src/App.css',
 'frontend/src/App.jsx',
 'frontend/src/index.css',
 'frontend/src/main.jsx',
 'backend/src/controllers/auth.controller.js',
 'backend/src/controllers/gemini.controller.js',
 'backend/src/controllers/message.controller.js',
 'backend/src/lib/cloudinary.js',
 'backend/src/lib/db.js',
 'backend/src/lib/socket.js',
 'backend/src/lib/utils.js',
 'backend/src/middlewares/auth.middleware.js',
 'backend/src/models/message.model.js',
 'backend/src/models/user.model.js',
 'backend/src/routes/auth.route.js',
 'ba

In [ ]:
def read_repo_file(repo_name: str, file_path: str) -> str:
    repo = github.get_repo(repo_name)
    file_content = repo.get_contents(file_path)

    # Decode file content (base64 → string)
    content = file_content.decoded_content.decode("utf-8", errors="ignore")
    return content

In [ ]:
display(Markdown(read_repo_file('rohitratannagar/talksy', 'package.json')))

{
  "name": "talksy",
  "version": "1.0.0",
  "main": "index.js",
  "scripts": {
    "build": "npm install --prefix backend && npm install --prefix frontend && npm run build --prefix frontend",
    "start": "npm run start --prefix backend"
  },
  "keywords": [],
  "author": "Rohit Ratan Nagar",
  "license": "ISC",
  "description": ""
}


In [ ]:
fetch_repo_files_json = {
    "name": "fetch_repo_files",
    "description": "Use this tool to fetch a list of all file paths in a GitHub repository.",
    "parameters": {
        "type": "object",
        "properties": {
            "repo_name": {
                "type": "string",
                "description": "The full GitHub repository name in the format 'username/repository'."
            }
        },
        "required": ["repo_name"],
        "additionalProperties": False
    }
}

In [ ]:
read_repo_file_json = {
    "name": "read_repo_file",
    "description": "Use this tool to read the contents of a specific file from a GitHub repository.",
    "parameters": {
        "type": "object",
        "properties": {
            "repo_name": {
                "type": "string",
                "description": "The full GitHub repository name in the format 'username/repository'."
            },
            "file_path": {
                "type": "string",
                "description": "The path of the file inside the repository to read."
            }
        },
        "required": ["repo_name", "file_path"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": fetch_repo_files_json},
         {"type": "function", "function": read_repo_file_json}]

In [ ]:
tools

[{'type': 'function',
  'function': {'name': 'fetch_repo_files',
   'description': 'Use this tool to fetch a list of all file paths in a GitHub repository.',
   'parameters': {'type': 'object',
    'properties': {'repo_name': {'type': 'string',
      'description': "The full GitHub repository name in the format 'username/repository'."}},
    'required': ['repo_name'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'read_repo_file',
   'description': 'Use this tool to read the contents of a specific file from a GitHub repository.',
   'parameters': {'type': 'object',
    'properties': {'repo_name': {'type': 'string',
      'description': "The full GitHub repository name in the format 'username/repository'."},
     'file_path': {'type': 'string',
      'description': 'The path of the file inside the repository to read.'}},
    'required': ['repo_name', 'file_path'],
    'additionalProperties': False}}}]

In [ ]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "fetch_repo_files":
            result = fetch_repo_files(**arguments)
        elif tool_name == "read_repo_file":
            result = read_repo_file(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
repo = github.get_repo(f"{github_username}/talksy")
repo.get_contents("")

[ContentFile(path=".gitignore"),
 ContentFile(path="README.md"),
 ContentFile(path="backend"),
 ContentFile(path="frontend"),
 ContentFile(path="package-lock.json"),
 ContentFile(path="package.json")]

In [ ]:
gemini  = OpenAI(api_key=gemini_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "openai/gpt-oss-120b"


In [ ]:
system_prompt = """ You are a Code Review AI.

Your task is to talk normally with user
Your task is to review GitHub repositories and help improve code quality.

You can use tools to:
- Get the list of files in a repository
- Read file contents

Your goals are to:
- Find bugs or logical mistakes
- Suggest improvements
- Improve readability and structure
- Point out bad practices

How to work:
- First, understand the project structure.
- Focus on important source files.
- Ignore irrelevant files like node_modules, build, or dist.
- Do not guess or hallucinate missing code.

When giving feedback:
- Mention the file name.
- Clearly explain the problem.
- Suggest a fix or improvement.
- Keep explanations short and clear.

Rules:
- Only review files you have seen.
- Use tools only when necessary.
- Be helpful and precise.

Act like a calm, experienced developer reviewing code. """

In [ ]:
github_username = "rohitratannagar"
repo_name = "talksy"

In [ ]:
user_prompt = f"""Please review my GitHub repository.

GitHub Username: {github_username}
Repository Name: {repo_name}

Your task:
1. Fetch the repository files.
2. Identify important source files.
3. Review the code for:
   - Bugs
   - Logical errors
   - Bad practices
   - Poor structure
4. Suggest improvements with clear explanations.

Focus only on files you can access.
Ignore build folders and dependencies.
Be concise and technical."""


In [ ]:
def chat(message, history):

    history = [{"role": h["role"], "content": h["content"]} for h in history]

    # Start with the system prompt
    messages_for_llm = [
        {"role": "system", "content": system_prompt}
    ]

    # Add cleaned history directly
    messages_for_llm.extend(history)

    # Add the current user's message
    messages_for_llm.append({"role": "user", "content": message})

    done = False
    while not done:
        response = gemini.chat.completions.create(
            model=model_name,
            messages=messages_for_llm,
            tools=tools
        )

        choice = response.choices[0]
        finish_reason = choice.finish_reason
        llm_response_message = choice.message

        if finish_reason == "tool_calls":
            # Add assistant tool-call message
            messages_for_llm.append({
                "role": "assistant",
                "content": llm_response_message.content,
                "tool_calls": llm_response_message.tool_calls,
            })

            tool_results = handle_tool_calls(llm_response_message.tool_calls)
            messages_for_llm.extend(tool_results)
        else:
            done = True

    return llm_response_message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1e9de8a1854eef232d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Tool called: fetch_repo_files
Tool called: read_repo_file
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://1e9de8a1854eef232d.gradio.live


In [128]:
!git init


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [129]:
!git branch -m main

In [130]:
!git add GitHub_Repo_Reviewer.ipynb


fatal: pathspec 'GitHub_Repo_Reviewer.ipynb' did not match any files


In [124]:
!git commit -m "uploading from colab"

[main (root-commit) deebaeb] uploading from colab
 22 files changed, 51090 insertions(+)
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 create mode 100644 .config/config_sentinel
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/default_configs.db
 create mode 100644 .config/gce
 create mode 100644 .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
 create mode 100644 .config/logs/2026.01.16/14.23.31.981136.log
 create mode 100644 .config/logs/2026.01.16/14.24.03.314209.log
 create mode 100644 .config/logs/2026.01.16/14.24.13.071214.log
 create mode 100644 .config/logs/2026.01.16/14.24.18.954466.log
 create mode 100644 .config/logs/2026.01.16/14.24.28.646070.log
 create mode 100644 .config/logs/2026.01.16/14.24.29.392089.log
 create mode 100644 .gradio/certificate.pem
 create mod

In [123]:
!git config --global user.email "rohitratannagar2003@gmail.com"
!git config --global user.name "rohitratannagar"

In [125]:
!git reset Head~1


fatal: ambiguous argument 'Head~1': unknown revision or path not in the working tree.
Use '--' to separate paths from revisions, like this:
'git <command> [<revision>...] -- [<file>...]'


In [126]:
!git status


On branch main
nothing to commit, working tree clean


In [127]:
!rm -rf .git